In [13]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.utils import shuffle

In [14]:
text = pd.read_csv('../data/ad.data', header = None)
text = shuffle(text)
text[:5]

/root/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3,4,5,6,7,8,9,...,1549,1550,1551,1552,1553,1554,1555,1556,1557,1558
119,?,?,?,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
1866,?,?,?,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nonad.
922,50,50,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nonad.
1286,?,?,?,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nonad.
223,60,88,1.4666,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.


In [15]:
labels = list(set(text[1558].values))
labels

['nonad.', 'ad.']

In [16]:
fold = int(round(text.shape[0]/10))

for num in range(10):
    w_org = np.array([np.zeros(text.shape[1]-5) for label in labels])
    w_avg = w_org.copy()
    
    if num == 0:
        train_data = text[fold:text.shape[0]]
    elif num == 9:
        train_data = text[:fold*9]
    else:
        train_data = pd.DataFrame(np.concatenate((text[:fold*num], text[fold*(num+1):]), axis=0))
            
    N_times = 3
    T_len = train_data.shape[0]
    step = N_times * T_len
    
    for t in range(N_times):
        y_true = []
        y_pred = []
        for train in train_data.index:

            row_data = train_data[train_data.index.isin([train])].values[0]

            w_list = [row_data[4:-1].dot(w.T) for w in w_org]

            pred_label = w_list.index(max(w_list))
            true_label = labels.index(row_data[-1])

            if pred_label != true_label:

                w_org[true_label] = w_org[true_label] + row_data[4:-1]
                w_org[pred_label] = w_org[pred_label] - row_data[4:-1]

                w_avg[true_label] = w_avg[true_label] + (step/(N_times * T_len)) * row_data[4:-1]
                w_avg[pred_label] = w_avg[pred_label] - (step/(N_times * T_len)) * row_data[4:-1]

            step = step -1

    pred_data=text[(fold*num):(fold*(num+1))]
    
    y_true = []
    y_pred = []
    for pred in pred_data.index:

        row_data = pred_data[pred_data.index.isin([pred])].values[0]

        w_list = [row_data[4:-1].dot(w.T) for w in w_avg]

        pred_label = w_list.index(max(w_list))

        true_label = labels.index(row_data[-1])

        y_true.append(labels[true_label])
        y_pred.append(labels[pred_label])
    print("Classification Report %s" % num + ":\n" + metrics.classification_report(y_true, y_pred))

Classification Report 1:
             precision    recall  f1-score   support

        ad.       0.72      0.86      0.78        42
     nonad.       0.98      0.95      0.96       286

avg / total       0.95      0.94      0.94       328

Classification Report 2:
             precision    recall  f1-score   support

        ad.       0.83      0.83      0.83        52
     nonad.       0.97      0.97      0.97       276

avg / total       0.95      0.95      0.95       328

Classification Report 3:
             precision    recall  f1-score   support

        ad.       0.76      0.89      0.82        47
     nonad.       0.98      0.95      0.97       281

avg / total       0.95      0.95      0.95       328

Classification Report 4:
             precision    recall  f1-score   support

        ad.       0.80      0.88      0.84        42
     nonad.       0.98      0.97      0.98       286

avg / total       0.96      0.96      0.96       328

Classification Report 5:
             pr